In [1]:
from pathlib import Path

CACHE_DIR = Path("data/cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

to_delete = []
to_delete += list(CACHE_DIR.glob("hw3_contract_ratings_*.csv"))
to_delete += list(CACHE_DIR.glob("hw3_benchmark_*.csv"))
to_delete += list(CACHE_DIR.glob("hw3_plan_county_year_enrollment_*.csv"))
to_delete += list(CACHE_DIR.glob("hw3_penetration_*.csv"))
to_delete += list(CACHE_DIR.glob("hw3_service_area_*.csv"))

for p in sorted(set(to_delete)):
    try:
        p.unlink()
        print("deleted", p)
    except Exception:
        pass

deleted data/cache/hw3_benchmark_2010.csv
deleted data/cache/hw3_benchmark_2011.csv
deleted data/cache/hw3_contract_ratings_2010.csv
deleted data/cache/hw3_contract_ratings_2011.csv
deleted data/cache/hw3_penetration_2010.csv
deleted data/cache/hw3_penetration_2011.csv
deleted data/cache/hw3_plan_county_year_enrollment_2010.csv
deleted data/cache/hw3_plan_county_year_enrollment_2011.csv
deleted data/cache/hw3_plan_county_year_enrollment_2012.csv
deleted data/cache/hw3_service_area_2010.csv
deleted data/cache/hw3_service_area_2011.csv
deleted data/cache/hw3_service_area_2012.csv


In [2]:
from __future__ import annotations

import re
import json
import io
import warnings
from pathlib import Path
from typing import Optional, Iterable

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 240)
pd.set_option("display.width", 200)

YEARS = list(range(2010, 2016))

HWK3_ROOT = Path.cwd()
CACHE_DIR = HWK3_ROOT / "data" / "cache"
PROCESSED_DIR = HWK3_ROOT / "data" / "processed"
CACHE_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

OUT_PLAN_COUNTY_YEAR = PROCESSED_DIR / "hw3_plan_county_year_2010_2015.csv"
OUT_PLAN_YEAR        = PROCESSED_DIR / "hw3_plan_year_2010_2015.csv"
OUT_CONTRACT_RATINGS = PROCESSED_DIR / "hw3_contract_ratings_2010_2015.csv"
OUT_RATING_DIST      = PROCESSED_DIR / "hw3_rating_distribution_2010_2015.csv"

FIXMAP_PATH = CACHE_DIR / "hw3_fixmap.json"
if not FIXMAP_PATH.exists():
    raise FileNotFoundError(f"Missing {FIXMAP_PATH}. Run your probe notebook first.")

FIXMAP = json.loads(FIXMAP_PATH.read_text())

CANDIDATE_MA_ROOTS = [
    Path("/scion/5261/econ470001/ma-data/ma"),
    Path("/home/rpat638/econ470/a0/work/ma-data/ma"),
    HWK3_ROOT.parent / "ma-data" / "ma",
]

def pick_existing(paths: list[Path]) -> Path:
    for p in paths:
        if p.exists():
            return p
    raise FileNotFoundError("No MA_ROOT found:\n" + "\n".join(map(str, paths)))

MA_ROOT = pick_existing(CANDIDATE_MA_ROOTS)

ENROLL_DIR = MA_ROOT / "enrollment" / "Extracted Data"
SAREA_DIR  = MA_ROOT / "service-area" / "Extracted Data"
PEN_DIR    = MA_ROOT / "penetration" / "Extracted Data"
STARS_DIR  = MA_ROOT / "star-ratings" / "Extracted Star Ratings"
BENCH_DIR  = MA_ROOT / "benchmarks"

TERRITORIES = {"VI", "PR", "MP", "GU", "AS"}

print("MA_ROOT:", MA_ROOT)
print("FIXMAP:", FIXMAP_PATH)
print("Outputs:", PROCESSED_DIR)

MA_ROOT: /scion/5261/econ470001/ma-data/ma
FIXMAP: /home/rpat638/econ470/a0/work/hwk3/data/cache/hw3_fixmap.json
Outputs: /home/rpat638/econ470/a0/work/hwk3/data/processed


In [3]:
def normalize_columns(cols: Iterable[str]) -> list[str]:
    out = []
    seen = {}
    for c in cols:
        c0 = str(c).strip().lower()
        c0 = re.sub(r"[^\w]+", "_", c0)
        c0 = re.sub(r"_+", "_", c0).strip("_")
        if c0 == "":
            c0 = "col"
        if c0 in seen:
            seen[c0] += 1
            c0 = f"{c0}_{seen[c0]}"
        else:
            seen[c0] = 0
        out.append(c0)
    return out

def read_csv_safe(path: Path, **kwargs) -> pd.DataFrame:
    engine = kwargs.get("engine", None)
    if engine == "python":
        kwargs.pop("low_memory", None)
    else:
        kwargs.setdefault("low_memory", False)
    kwargs.setdefault("on_bad_lines", "skip")

    for enc in ["utf-8", "latin-1", "cp1252"]:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, **kwargs)

def digits_only(x) -> str:
    return re.sub(r"\D", "", "" if pd.isna(x) else str(x))

def digits_n(x, n: int) -> Optional[str]:
    d = digits_only(x)
    if d == "":
        return None
    if len(d) > n:
        d = d[-n:]
    return d.zfill(n)

def clean_plan_id(x) -> Optional[str]:
    return digits_n(x, 3)

def clean_fips(x) -> Optional[str]:
    d = digits_n(x, 5)
    if d is None:
        return None
    return d if len(d) == 5 else None

def clean_ssa(x) -> Optional[str]:
    d = digits_n(x, 5)
    if d is None:
        return None
    return d if len(d) == 5 else None

def _as_series(x):
    if isinstance(x, pd.DataFrame):
        return x.iloc[:, 0]
    return x

def to_float_series(x) -> pd.Series:
    x = _as_series(x)
    s = x.astype(str)
    s = s.str.replace(",", "", regex=False)
    s = s.str.replace("*", "", regex=False)
    s = s.str.strip()
    num = s.str.extract(r"(\d+(?:\.\d+)?)", expand=False)
    return pd.to_numeric(num, errors="coerce")

def clean_contractid_series(x: pd.Series) -> pd.Series:
    s = _as_series(x).astype(str).str.strip()
    s = s.str.extract(r"([A-Z]\d{4,5})", expand=False)
    return s

def dedupe_cols(df: pd.DataFrame) -> pd.DataFrame:
    if df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()].copy()
    return df

def standardize_contractid(df: pd.DataFrame) -> pd.DataFrame:
    df = dedupe_cols(df)
    cols = list(df.columns)

    contract_cols = [c for c in cols if "contract" in c]
    base = contract_cols[0] if contract_cols else cols[0]

    contractid = clean_contractid_series(df[base])

    drop_cols = list(dict.fromkeys(contract_cols + ([base] if base not in contract_cols else [])))
    df2 = df.drop(columns=[c for c in drop_cols if c in df.columns]).copy()

    df2.insert(0, "contractid", contractid)
    df2 = dedupe_cols(df2)
    df2 = df2.dropna(subset=["contractid"]).copy()
    return df2

In [4]:
def find_enrollment_contract_file(year: int) -> Path:
    for p in [ENROLL_DIR / f"CPSC_Contract_Info_{year}_01.csv", ENROLL_DIR / f"CPSC_Contract_Info_{year}_1.csv"]:
        if p.exists():
            return p
    raise FileNotFoundError(f"Missing contract info for {year}")

def find_enrollment_month_files(year: int) -> list[Path]:
    files = sorted(ENROLL_DIR.glob(f"CPSC_Enrollment_Info_{year}_*.csv"))
    if not files:
        raise FileNotFoundError(f"Missing enrollment months for {year}")
    return files

def find_service_area_file(year: int) -> Optional[Path]:
    for p in [SAREA_DIR / f"MA_Cnty_SA_{year}_01.csv", SAREA_DIR / f"MA_Cnty_SA_{year}01.csv", SAREA_DIR / f"MA_Cnty_SA_{year}.csv"]:
        if p.exists():
            return p
    hits = sorted(SAREA_DIR.glob(f"*{year}*SA*csv"))
    return hits[0] if hits else None

def find_penetration_month_files(year: int) -> list[Path]:
    files = sorted(PEN_DIR.glob(f"State_County_Penetration_MA_{year}_*.csv"))
    if not files:
        raise FileNotFoundError(f"Missing penetration months for {year}")
    return files

In [5]:
def load_contract_info(year: int) -> pd.DataFrame:
    p = find_enrollment_contract_file(year)
    df = read_csv_safe(p, dtype=str)
    df.columns = normalize_columns(df.columns)
    df = dedupe_cols(df)

    rename_map = {
        "contract_id": "contractid",
        "contract_number": "contractid",
        "plan_id": "planid",
        "organization_type": "org_type",
        "plan_type": "plan_type",
        "offers_part_d": "partd",
        "snp_plan": "snp",
        "eghp": "eghp",
        "organization_name": "org_name",
        "organization_marketing_name": "org_marketing_name",
        "plan_name": "plan_name",
        "parent_organization": "parent_org",
    }
    for k, v in rename_map.items():
        if k in df.columns:
            df = df.rename(columns={k: v})

    df["contractid"] = df["contractid"].astype(str).str.strip()
    df["planid"] = df["planid"].apply(clean_plan_id)

    for c in ["snp", "partd", "plan_type", "org_type"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    df = df.dropna(subset=["contractid", "planid"])

    if "snp" in df.columns:
        df = df[df["snp"].str.lower().isin(["no", "n", "0", "false"])]

    planid_num = pd.to_numeric(df["planid"], errors="coerce")
    df = df[~(planid_num.between(800, 899, inclusive="both"))]

    if "plan_type" in df.columns:
        df = df[~df["plan_type"].astype(str).str.upper().str.contains("PDP", na=False)]

    keep = [c for c in [
        "contractid","planid","org_type","plan_type","partd","eghp",
        "org_name","org_marketing_name","plan_name","parent_org"
    ] if c in df.columns]
    return df[keep].drop_duplicates()

def build_plan_county_year_enrollment(year: int) -> pd.DataFrame:
    cache_path = CACHE_DIR / f"hw3_plan_county_year_enrollment_{year}.csv"
    if cache_path.exists():
        return pd.read_csv(cache_path, dtype=str)

    contract = load_contract_info(year)
    frames = []

    for p in find_enrollment_month_files(year):
        m = int(re.search(rf"{year}_(\d\d)\.csv$", p.name).group(1))
        dfm = read_csv_safe(p, dtype=str)
        dfm.columns = normalize_columns(dfm.columns)
        dfm = dedupe_cols(dfm)

        if "contract_number" in dfm.columns and "contractid" not in dfm.columns:
            dfm = dfm.rename(columns={"contract_number": "contractid"})
        if "contract_id" in dfm.columns and "contractid" not in dfm.columns:
            dfm = dfm.rename(columns={"contract_id": "contractid"})
        if "plan_id" in dfm.columns and "planid" not in dfm.columns:
            dfm = dfm.rename(columns={"plan_id": "planid"})
        if "fips_state_county_code" in dfm.columns and "fips" not in dfm.columns:
            dfm = dfm.rename(columns={"fips_state_county_code": "fips"})

        keep = [c for c in ["contractid","planid","fips","state","county","enrollment"] if c in dfm.columns]
        dfm = dfm[keep].copy()

        dfm["contractid"] = dfm["contractid"].astype(str).str.strip()
        dfm["planid"] = dfm["planid"].apply(clean_plan_id)
        dfm["fips"] = dfm["fips"].apply(clean_fips)
        dfm["enrollment"] = pd.to_numeric(dfm["enrollment"].astype(str).str.replace(",", "", regex=False), errors="coerce")

        dfm["state"] = dfm.get("state", None)
        dfm["county"] = dfm.get("county", None)
        if "state" in dfm.columns:
            dfm["state"] = dfm["state"].astype(str).str.strip()
        if "county" in dfm.columns:
            dfm["county"] = dfm["county"].astype(str).str.strip()

        dfm["month"] = m
        dfm = dfm.merge(contract[["contractid","planid"]], on=["contractid","planid"], how="inner")
        frames.append(dfm)

    df = pd.concat(frames, ignore_index=True)
    df = df.dropna(subset=["fips","contractid","planid"]).copy()
    df = df.sort_values(["state","county","contractid","planid","month"])
    df["fips"] = df.groupby(["state","county"])["fips"].transform(lambda x: x.ffill().bfill())

    out = (
        df.groupby(["contractid","planid","fips"], as_index=False)
          .agg(avg_enrollment=("enrollment","mean"))
    )
    dec = (
        df[df["month"] == 12]
        .groupby(["contractid","planid","fips"], as_index=False)
        .agg(dec_enrollment=("enrollment","sum"))
    )
    out = out.merge(dec, on=["contractid","planid","fips"], how="left")

    loc = (
        df.dropna(subset=["state","county"])
          .sort_values(["contractid","planid","fips","month"])
          .groupby(["contractid","planid","fips"], as_index=False)
          .tail(1)[["contractid","planid","fips","state","county"]]
    )
    out = out.merge(loc, on=["contractid","planid","fips"], how="left")

    out["year"] = year
    out = out.merge(contract, on=["contractid","planid"], how="left")

    out.to_csv(cache_path, index=False)
    return out

def load_service_area(year: int) -> pd.DataFrame:
    cache_path = CACHE_DIR / f"hw3_service_area_{year}.csv"
    if cache_path.exists():
        return pd.read_csv(cache_path, dtype=str)

    p = find_service_area_file(year)
    if p is None:
        return pd.DataFrame(columns=["contractid","fips"])

    df = read_csv_safe(p, dtype=str)
    df.columns = normalize_columns(df.columns)
    df = dedupe_cols(df)

    if "contract_id" in df.columns and "contractid" not in df.columns:
        df = df.rename(columns={"contract_id": "contractid"})
    if "contract" in df.columns and "contractid" not in df.columns:
        df = df.rename(columns={"contract": "contractid"})
    if "fips_state_county_code" in df.columns and "fips" not in df.columns:
        df = df.rename(columns={"fips_state_county_code": "fips"})

    df["contractid"] = df["contractid"].astype(str).str.strip()
    df["fips"] = df["fips"].apply(clean_fips)
    df = df.dropna(subset=["contractid","fips"]).drop_duplicates()

    out = df[["contractid","fips"]].copy()
    out.to_csv(cache_path, index=False)
    return out

def load_penetration(year: int) -> pd.DataFrame:
    cache_path = CACHE_DIR / f"hw3_penetration_{year}.csv"
    if cache_path.exists():
        return pd.read_csv(cache_path, dtype=str)

    frames = []
    for p in find_penetration_month_files(year):
        m = int(re.search(rf"{year}_(\d\d)\.csv$", p.name).group(1))
        dfm = read_csv_safe(p, dtype=str)
        dfm.columns = normalize_columns(dfm.columns)
        dfm = dedupe_cols(dfm)

        if "state_name" in dfm.columns:
            dfm = dfm.rename(columns={"state_name": "state"})
        if "county_name" in dfm.columns:
            dfm = dfm.rename(columns={"county_name": "county"})

        keep = [c for c in ["state","county","fips","ssa","eligibles","enrolled"] if c in dfm.columns]
        dfm = dfm[keep].copy()

        dfm["fips"] = dfm["fips"].apply(clean_fips)
        dfm["ssa"] = dfm["ssa"].apply(clean_ssa)
        dfm["eligibles"] = pd.to_numeric(dfm["eligibles"].astype(str).str.replace(",", "", regex=False), errors="coerce")
        dfm["enrolled"]  = pd.to_numeric(dfm["enrolled"].astype(str).str.replace(",", "", regex=False), errors="coerce")

        dfm["month"] = m
        frames.append(dfm)

    df = pd.concat(frames, ignore_index=True)
    df = df.dropna(subset=["fips"]).copy()
    df = df.sort_values(["state","county","month"])
    df["fips"] = df.groupby(["state","county"])["fips"].transform(lambda x: x.ffill().bfill())

    out = (
        df.groupby(["fips"], as_index=False)
          .agg(avg_eligibles=("eligibles","mean"),
               avg_enrolled=("enrolled","mean"),
               ssa=("ssa","last"))
    )
    out["year"] = year
    out.to_csv(cache_path, index=False)
    return out

In [6]:
def _read_star_csv(path: Path, skiprows: int) -> pd.DataFrame:
    df = read_csv_safe(path, skiprows=skiprows, header=0, dtype=str)
    if df.shape[1] == 1:
        df = read_csv_safe(path, skiprows=skiprows, header=0, dtype=str, sep=None, engine="python")
    df.columns = normalize_columns(df.columns)
    df = dedupe_cols(df)
    df = standardize_contractid(df)
    return df

def _domain_raw_score(domain: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in domain.columns if c != "contractid"]
    keep = []
    skip_tokens = [
        "contract","organization","org","parent","marketing","name","type",
        "overall","summary","domain","icon","performer","sanction","snp",
    ]
    for c in cols:
        cl = c.lower()
        if any(t in cl for t in skip_tokens):
            continue
        v = to_float_series(domain[c])
        n = int(v.notna().sum())
        if n < 150:
            continue
        frac = float(((v >= 1.0) & (v <= 5.0)).mean())
        if frac < 0.75:
            continue
        keep.append(c)

    if not keep:
        return domain[["contractid"]].drop_duplicates().assign(raw_score=np.nan)

    m = domain[["contractid"] + keep].copy()
    for c in keep:
        m[c] = to_float_series(m[c])
    m["raw_score"] = m[keep].mean(axis=1, skipna=True)
    return m[["contractid","raw_score"]].drop_duplicates()

def load_star_ratings(year: int) -> pd.DataFrame:
    cache_path = CACHE_DIR / f"hw3_contract_ratings_{year}.csv"
    if cache_path.exists():
        dfc = pd.read_csv(cache_path, dtype=str)
        need = {"contractid","year","partc_rating","raw_score","new_contract"}
        if need.issubset(set(dfc.columns)):
            return dfc

    yk = str(year)
    sfix = FIXMAP["stars"][yk]

    dom_path = Path(sfix["domain_path"])
    sum_path = Path(sfix["summary_path"])

    skip_dom = {2010:3, 2011:1, 2012:1, 2013:1, 2014:1, 2015:1}[year]
    skip_sum = {2010:1, 2011:1, 2012:1, 2013:1, 2014:1, 2015:1}[year]

    domain = _read_star_csv(dom_path, skiprows=skip_dom)
    summary = _read_star_csv(sum_path, skiprows=skip_sum)

    partc_col = sfix["picked_partc_col"]
    partd_col = sfix.get("picked_partd_col")
    partcd_col = sfix.get("picked_partcd_col")

    if year == 2015:
        cand = [c for c in summary.columns if "part_c" in c and "summary" in c and c != "contractid"]
        if cand:
            partc_col = cand[0]

    def safe_col(colname: Optional[str]) -> Optional[str]:
        if colname is None:
            return None
        return colname if colname in summary.columns else None

    partc_col = safe_col(partc_col)
    partd_col = safe_col(partd_col)
    partcd_col = safe_col(partcd_col)

    out = summary[["contractid"]].drop_duplicates().copy()
    out["year"] = year

    out["partc_rating"] = np.nan
    out["partd_rating"] = np.nan
    out["partcd_rating"] = np.nan
    out["new_contract"] = 0

    def flag_new(txt: pd.Series) -> pd.Series:
        t = txt.astype(str).str.lower()
        return t.str.contains("too new|new to rate|plan too new", regex=True, na=False)

    if partc_col is not None:
        tmp = summary[["contractid", partc_col]].copy()
        tmp["partc_rating"] = to_float_series(tmp[partc_col])
        out = out.merge(tmp[["contractid","partc_rating"]], on="contractid", how="left")

        newf = summary[["contractid"]].assign(_new=flag_new(summary[partc_col]).values)
        newf = newf.groupby("contractid")["_new"].max().reset_index()
        out = out.merge(newf, on="contractid", how="left")
        out["new_contract"] = np.where(out["_new"].fillna(False), 1, out["new_contract"])
        out = out.drop(columns=["_new"])

    if partd_col is not None:
        tmp = summary[["contractid", partd_col]].copy()
        tmp["partd_rating"] = to_float_series(tmp[partd_col])
        out = out.merge(tmp[["contractid","partd_rating"]], on="contractid", how="left")

    if partcd_col is not None:
        tmp = summary[["contractid", partcd_col]].copy()
        tmp["partcd_rating"] = to_float_series(tmp[partcd_col])
        out = out.merge(tmp[["contractid","partcd_rating"]], on="contractid", how="left")

    raw = _domain_raw_score(domain)
    out = out.merge(raw, on="contractid", how="left")

    out.loc[out["new_contract"] == 1, ["partc_rating","partd_rating","partcd_rating","raw_score"]] = np.nan

    n_star = int(pd.to_numeric(out["partc_rating"], errors="coerce").notna().sum())
    n_raw  = int(pd.to_numeric(out["raw_score"], errors="coerce").notna().sum())
    print(f"stars {year}: contracts={out.shape[0]} partc_nonmiss={n_star} raw_nonmiss={n_raw}")

    out.to_csv(cache_path, index=False)
    return out

In [7]:
def load_benchmark(year: int) -> pd.DataFrame:
    cache_path = CACHE_DIR / f"hw3_benchmark_{year}.csv"
    if cache_path.exists():
        dfc = pd.read_csv(cache_path, dtype=str)
        if {"ssa","year"}.issubset(set(dfc.columns)):
            return dfc

    yk = str(year)
    b = FIXMAP["benchmarks"][yk]
    if b.get("benchmark_path") is None or b.get("sep") is None or b.get("start_line") is None:
        out = pd.DataFrame({"ssa": [], "risk_ab": [], "year": []})
        out.to_csv(cache_path, index=False)
        return out

    path = Path(b["benchmark_path"])
    sep = b["sep"]
    start = int(b["start_line"])

    lines = path.read_text(errors="ignore").splitlines()
    data = "\n".join(lines[start:])

    if sep == "whitespace":
        df = pd.read_csv(io.StringIO(data), sep=r"\s+", header=None, dtype=str, engine="python", on_bad_lines="skip")
    else:
        df = pd.read_csv(io.StringIO(data), sep=sep, header=None, dtype=str, engine="python", on_bad_lines="skip")

    if df.shape[1] < 2:
        out = pd.DataFrame({"ssa": [], "risk_ab": [], "year": []})
        out.to_csv(cache_path, index=False)
        return out

    df0 = df.iloc[:, 0].astype(str).str.strip()
    ssa = df0.apply(clean_ssa)

    risk_ab = pd.Series(np.nan, index=df.index, dtype="float64")
    if df.shape[1] >= 9:
        risk_ab = pd.to_numeric(df.iloc[:, 8].astype(str).str.replace(",", "", regex=False), errors="coerce")
    else:
        
        cands = []
        for j in range(1, df.shape[1]):
            v = pd.to_numeric(df.iloc[:, j].astype(str).str.replace(",", "", regex=False), errors="coerce")
            if int(v.notna().sum()) < 1000:
                continue
            med = float(v.median())
            if 200 <= med <= 2000:
                cands.append((j, med))
        if cands:
            j = sorted(cands, key=lambda t: (t[0], t[1]))[-1][0]
            risk_ab = pd.to_numeric(df.iloc[:, j].astype(str).str.replace(",", "", regex=False), errors="coerce")

    out = pd.DataFrame({"ssa": ssa, "risk_ab": risk_ab, "year": year})
    out = out.dropna(subset=["ssa"]).drop_duplicates(subset=["ssa"])
    out.to_csv(cache_path, index=False)
    print("benchmark", year, "rows", out.shape[0])
    return out

In [8]:
all_rows = []
all_contract = []

for year in YEARS:
    print("\n" + "=" * 70)
    print("YEAR", year)

    plan = build_plan_county_year_enrollment(year)
    sa   = load_service_area(year)
    pen  = load_penetration(year)
    stars = load_star_ratings(year)
    bench = load_benchmark(year)

    print("enrollment rows", plan.shape[0])
    print("service area pairs", sa.shape[0])
    print("penetration counties", pen.shape[0])
    print("contract ratings rows", stars.shape[0])
    print("benchmark counties", bench.shape[0])

    df = plan.merge(sa.drop_duplicates(), on=["contractid","fips"], how="inner")
    if "state" in df.columns:
        df = df[~df["state"].isin(TERRITORIES)].copy()

    df = df.merge(pen[["fips","avg_eligibles","avg_enrolled","ssa"]], on="fips", how="left")

    df = df.merge(
        stars[["contractid","partc_rating","partd_rating","partcd_rating","raw_score","new_contract"]],
        on="contractid",
        how="left"
    )

    
    partd_no = df.get("partd", "").astype(str).str.strip().str.lower().isin(["no","n","0","false"])
    has_partcd = pd.to_numeric(df["partcd_rating"], errors="coerce").notna()

    df["Star_Rating"] = np.where(
        partd_no,
        pd.to_numeric(df["partc_rating"], errors="coerce"),
        np.where(has_partcd, pd.to_numeric(df["partcd_rating"], errors="coerce"), pd.to_numeric(df["partc_rating"], errors="coerce"))
    )

    df = df.merge(bench[["ssa","risk_ab"]], on="ssa", how="left")

    df["avg_enrollment"] = pd.to_numeric(df["avg_enrollment"], errors="coerce")
    df["dec_enrollment"] = pd.to_numeric(df.get("dec_enrollment", np.nan), errors="coerce")
    df["enrollment_used"] = df["dec_enrollment"].where(df["dec_enrollment"].notna(), df["avg_enrollment"])

    df["avg_enrolled"] = pd.to_numeric(df["avg_enrolled"], errors="coerce")
    df["mkt_share"] = df["enrollment_used"] / df["avg_enrolled"]
    df.loc[~np.isfinite(df["mkt_share"]), "mkt_share"] = np.nan

    df["hmo"] = df.get("plan_type", "").astype(str).str.upper().str.contains("HMO").astype(int)
    df["partd_ind"] = (~partd_no).astype(int)
    df["plan_key"] = df["contractid"].astype(str) + "-" + df["planid"].astype(str)
    df["year"] = year

    keep = [
        "year","contractid","planid","plan_key","fips","state","county",
        "enrollment_used","avg_enrollment","dec_enrollment","avg_enrolled","avg_eligibles","mkt_share",
        "Star_Rating","raw_score","new_contract",
        "partd","partd_ind","plan_type","hmo","org_type",
        "ssa","risk_ab",
    ]
    keep = [c for c in keep if c in df.columns]
    df = df[keep].copy()

    print("final rows", df.shape[0])
    print("Star_Rating non-missing", int(pd.to_numeric(df["Star_Rating"], errors="coerce").notna().sum()))
    print("raw_score non-missing", int(pd.to_numeric(df["raw_score"], errors="coerce").notna().sum()))
    print("mkt_share non-missing", int(pd.to_numeric(df["mkt_share"], errors="coerce").notna().sum()))

    all_rows.append(df)
    all_contract.append(stars.assign(year=year))

plan_county_year = pd.concat(all_rows, ignore_index=True)
contract_ratings = pd.concat(all_contract, ignore_index=True)

plan_county_year.to_csv(OUT_PLAN_COUNTY_YEAR, index=False)
contract_ratings.to_csv(OUT_CONTRACT_RATINGS, index=False)

py = plan_county_year.copy()
py["enrollment_used"] = pd.to_numeric(py["enrollment_used"], errors="coerce")
plan_year = (
    py.groupby(["year","contractid","planid"], as_index=False)
      .agg(total_enrollment=("enrollment_used","sum"),
           Star_Rating=("Star_Rating","first"),
           raw_score=("raw_score","first"),
           partd=("partd","first"),
           hmo=("hmo","first"),
           partd_ind=("partd_ind","first"))
)
plan_year.to_csv(OUT_PLAN_YEAR, index=False)

cr = contract_ratings.copy()
cr["partc_rating"] = pd.to_numeric(cr["partc_rating"], errors="coerce")
dist = (
    cr.dropna(subset=["partc_rating"])
      .groupby(["year","partc_rating"])
      .size()
      .reset_index(name="n_contracts")
      .rename(columns={"partc_rating": "Star_Rating"})
      .sort_values(["year","Star_Rating"])
)
dist.to_csv(OUT_RATING_DIST, index=False)

print("\nSaved")
print("plan_county_year:", OUT_PLAN_COUNTY_YEAR, plan_county_year.shape)
print("plan_year:", OUT_PLAN_YEAR, plan_year.shape)
print("contract_ratings:", OUT_CONTRACT_RATINGS, contract_ratings.shape)
print("rating_dist:", OUT_RATING_DIST, dist.shape)


YEAR 2010
stars 2010: contracts=708 partc_nonmiss=0 raw_nonmiss=0
benchmark 2010 rows 3247
enrollment rows 675013
service area pairs 436236
penetration counties 3280
contract ratings rows 708
benchmark counties 3247
final rows 109944
Star_Rating non-missing 0
raw_score non-missing 0
mkt_share non-missing 86786

YEAR 2011
stars 2011: contracts=575 partc_nonmiss=0 raw_nonmiss=507
benchmark 2011 rows 2161
enrollment rows 507548
service area pairs 381465
penetration counties 3228
contract ratings rows 575
benchmark counties 2161
final rows 67967
Star_Rating non-missing 0
raw_score non-missing 57318
mkt_share non-missing 54352

YEAR 2012
stars 2012: contracts=569 partc_nonmiss=0 raw_nonmiss=483
benchmark 2012 rows 3246
enrollment rows 498019
service area pairs 374396
penetration counties 3224
contract ratings rows 569
benchmark counties 3246
final rows 67206
Star_Rating non-missing 0
raw_score non-missing 59512
mkt_share non-missing 52887

YEAR 2013
stars 2013: contracts=578 partc_nonmiss=

In [9]:
df = plan_county_year.copy()
df["Star_Rating"] = pd.to_numeric(df["Star_Rating"], errors="coerce")
df["raw_score"] = pd.to_numeric(df["raw_score"], errors="coerce")
df["mkt_share"] = pd.to_numeric(df["mkt_share"], errors="coerce")
df["enrollment_used"] = pd.to_numeric(df["enrollment_used"], errors="coerce")

print("\nRows by year")
print(df.groupby("year").size().to_string())

print("\nMissing Star_Rating by year")
print(df.groupby("year")["Star_Rating"].apply(lambda x: int(x.isna().sum())).to_string())

print("\nMissing raw_score by year")
print(df.groupby("year")["raw_score"].apply(lambda x: int(x.isna().sum())).to_string())

print("\nStar rating distribution (plan-county rows, not unique contracts)")
print(
    df.dropna(subset=["Star_Rating"])
      .groupby(["year","Star_Rating"])
      .size()
      .unstack("Star_Rating")
      .fillna(0)
      .astype(int)
      .to_string()
)

d2010 = df[df["year"] == 2010].copy()
print("\n2010 checks")
print("2010 Star_Rating non-missing", int(d2010["Star_Rating"].notna().sum()))
print("2010 raw_score non-missing", int(d2010["raw_score"].notna().sum()))


Rows by year
year
2010    109944
2011     67967
2012     67206
2013     67739
2014     62211
2015     65403

Missing Star_Rating by year
year
2010    109944
2011     67967
2012     67206
2013     67739
2014     62211
2015     65403

Missing raw_score by year
year
2010    109944
2011     10649
2012      7694
2013      2663
2014     62211
2015     65403

Star rating distribution (plan-county rows, not unique contracts)
Empty DataFrame
Columns: []
Index: []

2010 checks
2010 Star_Rating non-missing 0
2010 raw_score non-missing 0
